In [2]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import os
from synphot import SpectralElement, Empirical1D, SourceSpectrum, Observation
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import minimize, LinearConstraint

SPEC_PIPE_LOC = "{}/JPL/W12_Drops/spec_paper/Spec_pipeline".format(os.path.expanduser("~"))
sys.path.append(SPEC_PIPE_LOC)
os.environ['SPEC_PIPE_LOC'] = SPEC_PIPE_LOC
from Spec_pipeline import SDSS_Spec


sys.path.append("Gas_and_dust/fullspec_dust_models/")
sys.path.append("Gas_and_dust/")
from polWave import PolWave

from polWave_gas import PolWave_gas

In [3]:
data = np.loadtxt("pol_measurements.dat", usecols=[1,2])
p_measured = data[:,0]
p_unc = data[:,1]

In [4]:
#Read the observed spectrum
spec = SDSS_Spec("W0116-0505", 3.173, "spec-7046-56568-0425.fits")

In [6]:
#Load the bands.
def load_bands():
    #Load the filters.
    R_spec = np.loadtxt("M_SPECIAL_R.txt")
    I_bess = np.loadtxt("M_BESS_I.txt")
    v_high = np.loadtxt("v_HIGH.txt", skiprows=2)

    R_spec = R_spec[R_spec[:,1]>0.01]
    I_bess = I_bess[I_bess[:,1]>0.01]
    v_high = v_high[v_high[:,1]>0.01]

    #Transform the wavelengths to angstroms.
    R_spec[:,0] *= 10
    I_bess[:,0] *= 10
    v_high[:,0] *= 10

    v_high = v_high[(v_high[:,0]>4000.) & (v_high[:,0]<10000.)]
    I_bess = I_bess[(I_bess[:,0]>4000.) & (I_bess[:,0]<10000.)]
    R_spec = R_spec[(R_spec[:,0]>4000.) & (R_spec[:,0]<10000.)]

    Rbp= SpectralElement(Empirical1D, points=R_spec[:,0], lookup_table=R_spec[:,1]/100., keep_neg=False)
    Ibp= SpectralElement(Empirical1D, points=I_bess[:,0], lookup_table=I_bess[:,1]/100., keep_neg=False)
    vbp= SpectralElement(Empirical1D, points=v_high[:,0], lookup_table=v_high[:,1]/100., keep_neg=False)

    return [vbp, Rbp, Ibp]

In [7]:
bands = load_bands()

In [8]:
full_spec = SourceSpectrum(Empirical1D, points=spec.lam_obs, lookup_table=spec.flam, keep_neg=True)

In [9]:
def get_p_bb(dust_type, spec, bands, fw=True, bw=True):

    theta_angles = np.arange(0., 90.1, 5.)
    psi_angles = np.arange(0., 90.1, 5.)

    p_bb = np.zeros((len(bands), len(theta_angles), len(psi_angles)))
    p_bb_interp = list()

    if dust_type=="gas":
        model = PolWave_gas(fw=fw,bw=bw)
        for iband in range(len(bands)):
            p_bb_interp.append(model.p)
        #     for jtheta, theta in enumerate(theta_angles):
        #         th_aux = theta*np.ones(len(psi_angles))
        #         p_bb[iband, jtheta] = model.p((th_aux, psi_angles))

    else:
        model = PolWave(dust_type, folder="Gas_and_dust/fullspec_dust_models/", fw=fw, bw=bw)

        for iband, band in enumerate(bands):
            obs_I = Observation(full_spec, band)
            Ibb = obs_I.effstim(flux_unit='flam').value
            for jtheta, theta in enumerate(theta_angles):
                for kpsi, psi in enumerate(psi_angles):
                    th_aux = theta*np.ones(len(spec.lam_obs))
                    psi_aux = psi*np.ones(len(spec.lam_obs))
                    p_lam = model.p((spec.lam_rest.to(u.AA).value, th_aux, psi_aux))
            
                    Q_spec = SourceSpectrum(Empirical1D, points=spec.lam_obs, lookup_table=spec.flam * p_lam, keep_neg=True)

                    obs_Q = Observation(Q_spec, band)
                    Qbb = obs_Q.effstim(flux_unit='flam').value

                    p_bb[iband, jtheta, kpsi] = Qbb/Ibb
            p_bb_interp.append(RegularGridInterpolator((theta_angles, psi_angles), p_bb[iband]))
    return p_bb_interp
    

In [10]:
def chi2(x, p_bb_interp, bands, pm, pu):
    if x[0]<0 or x[0]>90. or x[1]<0. or x[1]>90.:
        return np.inf
    p_bb = np.zeros(len(bands))
    for iband in range(len(bands)):
        p_bb[iband] = p_bb_interp[iband](([x[0]], [x[1]]))
    return np.sum(((p_bb-pm)/pu)**2)
    

In [11]:
dust_types = ["gas","SMC","LMC","MW"]
fw = True
bw = True

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc)) 
    print(xopt)       

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310886372244
       x: [ 4.021e+01  5.915e+01]
     nit: 8
     jac: [ 5.492e-03 -4.338e-03]
    nfev: 26
    njev: 8
SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 754.0326495683184
       x: [ 6.198e+01  5.094e+01]
     nit: 12
     jac: [ 1.266e-03 -1.289e-03]
    nfev: 39
    njev: 12
LMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 891.6104171543943
       x: [ 5.917e+01  4.821e+01]
     nit: 11
     jac: [-1.541e-03  2.266e-03]
    nfev: 36
    njev: 11
MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 667.6529464495616
       x: [ 3.500e+01  1.206e+01]
     nit: 25
     jac: [ 1.010e+00 -3.288e-03]
    nfev: 94
    njev: 25


In [12]:
dust_types = ["gas","SMC","LMC","MW"]
fw = True
bw = False

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc)) 
    print(xopt)       

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310896442009
       x: [ 3.701e+01  5.447e+01]
     nit: 7
     jac: [ 2.117e-03 -1.230e-03]
    nfev: 23
    njev: 7
SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 774.1140329936595
       x: [ 5.881e+01  4.389e+01]
     nit: 14
     jac: [ 5.722e-04 -1.083e-03]
    nfev: 47
    njev: 14
LMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 887.7124373573126
       x: [ 5.295e+01  3.896e+01]
     nit: 7
     jac: [ 1.404e-03 -2.060e-03]
    nfev: 23
    njev: 7
MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 619.1419572961005
       x: [ 4.184e+01  2.335e+01]
     nit: 13
     jac: [-3.586e-04  5.493e-04]
    nfev: 42
    njev: 13


In [13]:
dust_types = ["gas","SMC","LMC","MW"]
fw = False
bw = True

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc))
    print(xopt)
    for iband in range(len(bands)):
        print(p_measured[iband], p_unc[iband], p_bb_interp[iband](([xopt.x[0]], [xopt.x[1]])))
    print()

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310896442009
       x: [ 3.701e+01  5.447e+01]
     nit: 7
     jac: [ 2.117e-03 -1.230e-03]
    nfev: 23
    njev: 7
0.0973 0.0037 [0.11300468]
0.1085 0.0023 [0.11300468]
0.1466 0.0041 [0.11300468]

SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 694.7015440378998
       x: [ 4.956e+01  6.123e+01]
     nit: 11
     jac: [-2.213e-04  1.144e-04]
    nfev: 35
    njev: 11
0.0973 0.0037 [0.14078039]
0.1085 0.0023 [0.09737138]
0.1466 0.0041 [0.05192698]

LMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 837.516264753161
       x: [ 5.613e+01  6.584e+01]
     nit: 12
     jac: [-2.823e-04  1.984e-04]
    nfev: 39
    njev: 12
0.0973 0.0037 [0.14253645]
0.1085 0.0023 [0.09312519]
0.1466 0.0041 [0.04260583]

MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 701.4986465318774
       x: [ 7